# Fit Gamma factor

> *This notebook estimates the gamma factor from a set of 5 μs-ALEX smFRET measurements.*

## What this notebook does?

According to [Lee 2005](http://dx.doi.org/10.1529/biophysj.104.054114) ([PDF](http://www.chem.ucla.edu/~michalet/papers/BJ2005.pdf), [SI PDF](http://www.chem.ucla.edu/~michalet/papers/BJ2005SI.pdf)), we estimate the $\gamma$-factor 
from Proximity Ratio (PR) and S values (with background, leakage and direct excitation correction) 
for a set of 5 μs-ALEX measurements.

The PR and S values are computed by the notebook

- [usALEX-5samples-PR-leakage-dir-ex-all-ph](usALEX-5samples-PR-leakage-dir-ex-all-ph.ipynb)

which is executed by [8-spots paper analysis](8-spots paper analysis.ipynb).

From [Lee 2005](http://dx.doi.org/10.1529/biophysj.104.054114) (equation 20), the following linear relation holds:

$$\frac{1}{S} = \Omega + \Sigma \cdot E_{PR}$$

Once $\Omega$ and $\Sigma$ are fitted, we can compute the $\gamma$-factor as ([equation 22](http://www.sciencedirect.com/science/article/pii/S0006349505733464#eq22)):

$$\gamma = (\Omega-1)/(\Omega + \Sigma-1)$$

$$\beta = \Omega + \Sigma - 1$$

The definition of $\beta$ based on physical parameters is:

$$ \beta = \frac{I_{A_{ex}}\sigma_{A_{ex}}^A}{I_{D_{ex}}\sigma_{D_{ex}}^D}$$

Note that, calling $S_\gamma$ the corrected S, the following relation holds:

$$ S_\gamma = (1 + \beta)^{-1}$$


## Import libraries

In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
import lmfit
from scipy.stats import linregress

## Computation

This notebook read data from the file:

In [ ]:
data_file = 'results/usALEX-5samples-PR-leakage-dir-ex-all-ph.csv'

In [ ]:
data = pd.read_csv(data_file).set_index('sample')
data

In [ ]:
data[['E_gauss_w', 'E_kde_w', 'S_gauss']]

In [ ]:
E_ref, S_ref = data.E_gauss_w, data.S_gauss

In [ ]:
res = linregress(E_ref, 1/S_ref)
slope, intercept, r_val, p_val, stderr = res

For more info see [`scipy.stats.linearregress`](http://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.stats.linregress.html).

In [ ]:
Sigma = slope 
Sigma

In [ ]:
Omega = intercept
Omega

[Pearson correlation coefficient](http://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient):

In [ ]:
r_val

[Coefficient of determination](http://en.wikipedia.org/wiki/Coefficient_of_determination) $R^2$:

In [ ]:
r_val**2

P-value (to test the null hypothesis that the slope is zero):

In [ ]:
p_val

Gamma computed from the previous fitted values:

In [ ]:
gamma = (Omega - 1)/(Omega + Sigma - 1)
'%.6f' % gamma

In [ ]:
with open('results/usALEX - gamma factor - all-ph.csv', 'w') as f:
    f.write('%.6f' % gamma)

In [ ]:
beta = Omega + Sigma - 1
'%.6f' % beta

In [ ]:
with open('results/usALEX - beta factor - all-ph.csv', 'w') as f:
    f.write('%.6f' % beta)

# Fit plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'  # for hi-dpi displays

sns.set_style('whitegrid')

In [ ]:
x = np.arange(0, 1, 0.01)
plt.plot(E_ref, 1/S_ref, 's', label='dsDNA samples')
plt.plot(x, intercept + slope*x, 'k', label='fit (slope = %.2f)' % slope)
plt.legend(loc=4)
plt.ylim(1, 2)
plt.xlabel('PR')
plt.ylabel('1/SR');